# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [4]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))
layer(torch.FloatTensor([1, 2, 3, 4, 5,6,7]))

tensor([-3., -2., -1.,  0.,  1.,  2.,  3.])

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [5]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [11]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [12]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [18]:
linear = MyLinear(5, 3)
print(linear.weight)

linear(torch.rand(2, 5))

Parameter containing:
tensor([[-0.0470,  3.4404,  0.0284],
        [-0.5516, -0.3267, -0.0415],
        [ 0.6444,  0.7150,  0.5598],
        [ 0.4184, -0.1816, -0.8392],
        [ 1.5778,  2.9411, -0.3489]], requires_grad=True)


tensor([[0.1997, 4.3156, 0.6216],
        [1.2237, 4.5207, 0.6360]])

我们可以[**使用自定义层直接执行前向传播计算**]。


In [20]:
linear(torch.rand(2, 5))

tensor([[0.8923, 5.0641, 0.8649],
        [0.0707, 3.8562, 1.1139]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [25]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


In [38]:
#练习一：设计一个接受输入并计算张量降维的层，它返回 yk=∑i,jWijkxixj 。

class Exercise1(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(input, input, output))

    def forward(self, X):
        print(X)
        print(self.weight)
        batch_size = X[0]
        X_i = X.unsqueeze(-1)# (batch_size, input_dim, 1)
        X_j = X.unsqueeze(-2)# (batch_size, 1, input_dim)
        print(f'xixj={(X_i*X_j).shape}')
        print(f'xi:{X_i},\nxj:{X_j}')
        print(f'weight:{self.weight.shape}')
        quadratic_terms = torch.einsum('bij,ijk->bk', X_i * X_j, self.weight)

        return quadratic_terms

'''
import torch
from torch import nn
import torch.nn.functional as F

class QuadraticLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # 初始化权重参数 W，形状为 (input_dim, input_dim, output_dim)
        self.weight = nn.Parameter(torch.randn(input_dim, input_dim, output_dim))
        # 初始化偏置参数
        self.bias = nn.Parameter(torch.randn(output_dim,))

    def forward(self, X):

        batch_size = X.shape[0]  # 获取批量大小
        # 扩展 X 以便进行广播计算
        X_i = X.unsqueeze(-1)  # (batch_size, input_dim, 1)
        X_j = X.unsqueeze(-2)  # (batch_size, 1, input_dim)
        # 计算所有组合 x_i * x_j 并与权重 W 相乘
        quadratic_terms = torch.einsum('bij,ijk->bk', X_i * X_j, self.weight)
        # 加上偏置并返回
        return quadratic_terms + self.bias
'''
# 测试
input_dim, output_dim = 4, 3
X = torch.rand(2, input_dim)  # 输入维度为 4，批量大小为 2
layer = Exercise1(input_dim, output_dim)
output = layer(X)
print(f"outputshape:{output.shape}")  # Use an f-string to print the shape
print(f"output:\n{output}") # Use an f-string to print the output tensor


tensor([[0.8496, 0.0747, 0.7134, 0.9193],
        [0.0827, 0.3759, 0.6460, 0.0294]])
Parameter containing:
tensor([[[-1.3716, -1.2201,  0.5644],
         [ 0.4686,  0.0359,  0.5512],
         [-1.7843,  0.3642, -1.0721],
         [ 1.5278,  0.7624, -0.3082]],

        [[ 1.6962, -0.1333,  0.1517],
         [-0.0707, -0.1057,  0.2990],
         [ 2.9393, -1.6836, -0.4789],
         [-2.4966,  0.7081, -1.3390]],

        [[-0.0251, -1.1284,  1.9158],
         [-0.1191,  3.0230,  0.1689],
         [ 1.0323, -1.0270,  0.1542],
         [ 0.2380,  0.4777,  0.0718]],

        [[-0.5592,  2.7877, -0.0919],
         [-0.2329,  1.2463, -0.1531],
         [ 1.1700, -0.9133, -0.4053],
         [ 0.8931, -0.0502, -0.0064]]], requires_grad=True)
xixj=torch.Size([2, 4, 4])
xi:tensor([[[0.8496],
         [0.0747],
         [0.7134],
         [0.9193]],

        [[0.0827],
         [0.3759],
         [0.6460],
         [0.0294]]]),
xj:tensor([[[0.8496, 0.0747, 0.7134, 0.9193]],

        [[0.0827, 0.37

[Discussions](https://discuss.d2l.ai/t/1835)
